### Settings and imports

In [1]:
%matplotlib notebook

In [2]:
import masserstein 

In [3]:
import os

from masserstein import Spectrum, NMRSpectrum
from masserstein import estimate_proportions

from matplotlib import pyplot as plt

import numpy as np

import pulp

import pickle

import seaborn as sns
from textwrap import wrap

from pathlib import Path

### Loading the data

In [4]:
official_names = ['Pinene', 'Benzyl benzoate']

In [5]:
os.getcwd()

'/home/basia/Documents/spectroscopy/new_magnetstein_data/experiment_1_intensity_difference'

In [6]:
mix = np.loadtxt(os.getcwd()+'/benzyl_benzoate_pinene_duza_roznica_intensywnosci_pikow.csv', skiprows=0, usecols=range(2), 
                      delimiter='\t',dtype=float)
ppm = mix[:,0]
mix = mix[:,1]

comp = np.loadtxt(os.getcwd()+'/library.csv', skiprows=1, usecols=[0, 1, 2],
                      delimiter='\t',dtype=float)
ppm_components = comp[:,0]
components = comp[:,1:]

In [7]:
components_ints = []
names = []
for i in range(components.shape[1]):
    components_ints.append(components[:,i])
    names.append('comp'+str(i))

In [8]:
protons = [16, 12]
mass_proportions = [0.0603, 0.9397]
molar_masses = [136.24, 212.25]

In [9]:
spectra = []
for i, comp_ints in enumerate(components_ints):
    spectra.append(NMRSpectrum(confs=list(zip(ppm_components, comp_ints)), protons = protons[i]))

In [10]:
mix = NMRSpectrum(confs=list(zip(ppm,mix)))

In [11]:
# mix.plot(profile=True)
# spectra[0].plot(profile=True)
# spectra[1].plot(profile=True)

### Ground truth

In [12]:
protons = [16, 12]
mass_proportions = [0.0603, 0.9397]
molar_masses = [136.24, 212.25]

In [13]:
molar_proportions = [mass_prop/mol_mass for mass_prop, mol_mass in zip(mass_proportions, molar_masses)]
molar_proportions = [mol_prop/sum(molar_proportions) for mol_prop in molar_proportions]

In [14]:
molar_proportions

[0.09088457406472417, 0.9091154259352758]

In [15]:
real_visible_proportions = [prot*prop for prot, prop in zip(protons, molar_proportions)]
real_visible_proportions = [prop/sum(real_visible_proportions) for prop in real_visible_proportions]

In [16]:
real_visible_proportions

[0.11761626406540927, 0.8823837359345907]

In [17]:
mass_proportions

[0.0603, 0.9397]

### Preprocessing

In [19]:
def make_nonnegative(spectrum):
    confs = np.array(spectrum.confs)
    neg = confs[:,1]<0
    confs[:,1][neg]=0
    new_confs = list(zip(confs[:,0],confs[:,1]))
    result = NMRSpectrum(confs=new_confs)
    return(result)

In [20]:
def common_mass_axis(list_of_spectra):
    all_mz = set()
    for sp in list_of_spectra:
        mz_set = set(np.array(sp.confs, np.dtype('float'))[:,0])
        all_mz = all_mz.union(mz_set)
    all_mz = np.array(sorted(all_mz))
    return all_mz

In [21]:
spectra_and_mixture = spectra + [mix]

In [24]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    sp2 = make_nonnegative(sp)
    sp2.sort_confs()
    sp2.merge_confs()
    sp2.normalize()
    preprocessed_spectra.append(sp2)

In [25]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [27]:
cma = common_mass_axis(spectra_and_mixture)

In [28]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    preprocessed_spectra.append(sp.resample(cma))

In [29]:
spectra_and_mixture = preprocessed_spectra
del(preprocessed_spectra)

In [30]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    sp = make_nonnegative(sp)
    sp.sort_confs()
    sp.merge_confs()
    sp.normalize()
    preprocessed_spectra.append(sp)

In [31]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [32]:
for i, sp in enumerate(spectra):
    sp.protons = protons[i]

In [33]:
for i, name in enumerate(names):
    globals()[name] = spectra[i]

In [34]:
# mix.plot(profile=True)
# comp0.plot(profile=True)
# comp1.plot(profile=True)

### Finding best values of the parameters

In [35]:
import time
import pandas as pd
import pulp

In [36]:
lower_limit = 0.01
upper_limit = 0.3
step = 0.01

In [34]:
# list_of_estimation_results = []
# for kappa in np.arange(start=lower_limit, stop=upper_limit, step=step):
#     fixed_kappa = []
#     for kappa_prime in np.arange(start=lower_limit, stop=upper_limit, step=step):
#         start = time.time()
#         estimation = estimate_proportions(mix, [comp0, comp1], 
#                                    MTD=kappa, MTD_th=kappa_prime, verbose=False, solver=pulp.GUROBI(msg=False))
#         end = time.time()
#         fixed_kappa.append(estimation['proportions'])
#         print('Estimation for '+str(kappa)+' and '+str(kappa_prime)+' done')
#         print('It took: '+str(end-start)+' seconds.')
#         print(estimation['proportions'])
#     list_of_estimation_results.append(fixed_kappa)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10


Computing chunks: 100%|████████████████████████| 2/2 [00:00<00:00, 76260.07it/s]
Splitting the experimental spectrum into chunks: 131072it [00:00, 3446269.57it/s]
Deconvolving chunks:   0%|                                | 0/1 [00:00<?, ?it/s]/home/basia/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
Deconvolving chunks: 100%|███████████████████████| 1/1 [03:09<00:00, 189.12s/it]


Estimation for 0.01 and 0.01 done
It took: 189.27106976509094 seconds.
[0.004978539199000028, 0.6327822619960035]


Computing chunks: 100%|████████████████████████| 2/2 [00:00<00:00, 78398.21it/s]
Splitting the experimental spectrum into chunks: 131072it [00:00, 3591320.91it/s]
Deconvolving chunks: 100%|███████████████████████| 1/1 [03:12<00:00, 192.32s/it]


Estimation for 0.01 and 0.02 done
It took: 192.46171641349792 seconds.
[0.002270173790000013, 0.547106646037003]


...

Estimation for 0.29000000000000004 and 0.27 done
It took: 260.87420630455017 seconds.
[0.11975341765500067, 0.8702754283270049]


Computing chunks: 100%|████████████████████████| 2/2 [00:00<00:00, 73584.28it/s]
Splitting the experimental spectrum into chunks: 131072it [00:00, 3120545.22it/s]
Deconvolving chunks: 100%|███████████████████████| 1/1 [04:06<00:00, 246.28s/it]


Estimation for 0.29000000000000004 and 0.28 done
It took: 246.46186447143555 seconds.
[0.11957072391700066, 0.8704346193330049]


Computing chunks: 100%|████████████████████████| 2/2 [00:00<00:00, 84733.41it/s]
Splitting the experimental spectrum into chunks: 131072it [00:00, 3237685.82it/s]
Deconvolving chunks: 100%|███████████████████████| 1/1 [04:12<00:00, 252.95s/it]

Estimation for 0.29000000000000004 and 0.29000000000000004 done
It took: 253.10333514213562 seconds.
[0.11949059176000067, 0.8704987890740049]


In [37]:
# list_of_dataframes_with_results = []
# for component_number in range(len([comp0, comp1])):
#     results_for_component = pd.DataFrame(columns=np.arange(start=lower_limit, stop=upper_limit, step=step),
#                                    index=np.arange(start=lower_limit, stop=upper_limit, step=step))
#     for i in range(results_for_component.shape[0]):
#         for j in range(results_for_component.shape[1]):
#             results_for_component.iloc[i,j] = list_of_estimation_results[i][j][component_number]
    
#     list_of_dataframes_with_results.append(results_for_component)

In [38]:
# with open('results_for_different_kappas.pkl', 'wb') as f:
#     pickle.dump(list_of_dataframes_with_results, f)

#### Molar proportions

In [37]:
with open('results_for_different_kappas.pkl', 'rb') as f:
    list_of_dataframes_with_results = pickle.load(f)
#first coordinate: kappa,
#second coordinate: kappa_prime

In [38]:
temp = []
for nr_of_component, df in enumerate(list_of_dataframes_with_results):
    temp.append(df/protons[nr_of_component])
temp2 = []
for df in temp:
    temp2.append(df/sum(temp))
list_of_dataframes_with_results = temp2
del(temp)
del(temp2)

In [39]:
dataframes_ready_for_heatmap = []
for i, df in enumerate(list_of_dataframes_with_results):
    preprocessed_df = abs(df - molar_proportions[i])
    dataframes_ready_for_heatmap.append(preprocessed_df)

In [40]:
all_components_results_molar = sum(dataframes_ready_for_heatmap)

#### Visible proportions

In [41]:
with open('results_for_different_kappas.pkl', 'rb') as f:
    list_of_dataframes_with_results = pickle.load(f)
#first coordinate: kappa,
#second coordinate: kappa_prime

In [42]:
dataframes_ready_for_heatmap = []
for i, df in enumerate(list_of_dataframes_with_results):
    preprocessed_df = abs(df - real_visible_proportions[i])
    dataframes_ready_for_heatmap.append(preprocessed_df)

In [43]:
all_components_results_vis = sum(dataframes_ready_for_heatmap)

In [44]:
all_components_results_vis = all_components_results_vis.apply(pd.to_numeric, errors = 'coerce', axis=0)

#### Both

In [45]:
all_components_results_both = all_components_results_vis + all_components_results_molar

In [46]:
all_components_results_both = all_components_results_both.apply(pd.to_numeric, errors = 'coerce', axis=0)

In [47]:
all_components_results_both.min().min()

0.011008543679603217

In [48]:
all_components_results_both[0.08][0.09]

0.011008543679603217

### Saving preprocessed spectra

In [49]:
# for i, sp in enumerate(spectra_and_mixture):
#     try:
#         np.savetxt('preprocessed_'+str(names[i])+'.csv', np.array(sp.confs), delimiter=',')
#     except IndexError:
#         np.savetxt('preprocessed_mix.csv', np.array(sp.confs), delimiter=',')